# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-30-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-30-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-01 05:26:18,33.93911,67.709953,46498,1774,36788,7936.0,Afghanistan,119.445111,3.815218
1,NaN,NaN,NaN,Albania,2020-12-01 05:26:18,41.15330,20.168300,38182,810,18849,18523.0,Albania,1326.777399,2.121418
2,NaN,NaN,NaN,Algeria,2020-12-01 05:26:18,28.03390,1.659600,83199,2431,53809,26959.0,Algeria,189.730949,2.921910
3,NaN,NaN,NaN,Andorra,2020-12-01 05:26:18,42.50630,1.521800,6745,76,5873,796.0,Andorra,8729.696499,1.126761
4,NaN,NaN,NaN,Angola,2020-12-01 05:26:18,-11.20270,17.873900,15139,348,7851,6940.0,Angola,46.062425,2.298699


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,44503,44706,44988,45280,45490,45716,45839,45966,46215,46498
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,32196,32761,33556,34300,34944,35600,36245,36790,37625,38182
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,73774,74862,75867,77000,78025,79110,80168,81212,82221,83199


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1675,1687,1695,1712,1725,1737,1740,1752,1763,1774
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,685,699,716,735,743,753,771,787,798,810
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2255,2272,2294,2309,2329,2352,2372,2393,2410,2431


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35422,35934,35976,36122,36145,36232,36295,36709,36731,36788
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,15469,15842,16230,16666,17031,17352,17755,18152,18481,18849
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,48183,48794,49421,50070,50712,51334,51946,52568,53204,53809


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2597,2617,2634,2661,2686,2704,2716,2735,2751,2780
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,8131,8199,8269,8376,8473,8576,8603,8733,8820,8890
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1157,1160,1161,1167,1170,1170,1171,1173,1175,1178


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,39,39,39,39,41,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,84,84,84,84,98,98,98,98,98,98
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,10,10,10,10,10,10,10,10,11,11


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
648,1001.0,Autauga,Alabama,US,2020-12-01 05:26:18,32.539527,-86.644082,2780,42,0,2738.0,"Autauga, Alabama, US",4975.925826,1.510791
685,1075.0,Lamar,Alabama,US,2020-12-01 05:26:18,33.779950,-88.096680,689,8,0,681.0,"Lamar, Alabama, US",4990.945310,1.161103
686,1077.0,Lauderdale,Alabama,US,2020-12-01 05:26:18,34.901719,-87.656247,4223,54,0,4169.0,"Lauderdale, Alabama, US",4554.130854,1.278712


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,13541221,268045,5146319
India,9462809,137621,8889585
Brazil,6335878,173120,5597802


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,13541221,268045,5146319,8126857,2020-12-01 05:26:18,37.936303,-91.379001
India,9462809,137621,8889585,435603,2020-12-01 05:26:18,23.088275,81.806127
Brazil,6335878,173120,5597802,564956,2020-12-01 05:26:18,-12.669522,-48.480493
Russia,2275936,39491,1763493,472952,2020-12-01 05:26:18,54.546312,62.120860
France,2275016,52819,167913,2054284,2020-12-01 05:26:18,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1238752,21896,0
California,1230264,19173,0
Florida,999319,18597,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1238752,21896,0,1216856,2020-12-01 05:26:18,31.660643,-98.653069
California,1230264,19173,0,1211091,2020-12-01 05:26:18,37.843962,-120.728594
Florida,999319,18597,0,980722,2020-12-01 05:26:18,28.940755,-82.700744
Illinois,726304,12985,0,713319,2020-12-01 05:26:18,39.843618,-89.178330
New York,647980,34605,0,613375,2020-12-01 05:26:18,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,400919,7655,0
Illinois,Cook,306369,6669,0
Florida,Miami-Dade,229618,3835,0
Arizona,Maricopa,205658,3982,0
Texas,Harris,191513,3009,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,400919,7655,0,393264,2020-12-01 05:26:18,34.308284,-118.228241,6037.0
Illinois,Cook,306369,6669,0,299700,2020-12-01 05:26:18,41.841448,-87.816588,17031.0
Florida,Miami-Dade,229618,3835,0,225783,2020-12-01 05:26:18,25.611236,-80.551706,12086.0
Arizona,Maricopa,205658,3982,0,201676,2020-12-01 05:26:18,33.348359,-112.491815,4013.0
Texas,Harris,191513,3009,0,188504,2020-12-01 05:26:18,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,45966,36790,81212,6610,15087,141,1413375,133594,27892,275661,...,5511,72718,1,101760,1341,81890,10,2160,17589,9822
2020-11-29,46215,37625,82221,6712,15103,141,1418807,134768,27902,279708,...,5716,72870,1,102040,1343,83585,10,2177,17608,9822
2020-11-30,46498,38182,83199,6745,15139,141,1424533,135124,27912,282456,...,5857,73094,1,102394,1347,85647,10,2191,17647,9950


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,1752,787,2393,76,345,4,38322,2121,907,3018,...,75,608,0,892,35,704,1,615,357,275
2020-11-29,1763,798,2410,76,346,4,38473,2142,908,3105,...,76,608,0,894,35,717,1,617,357,275
2020-11-30,1774,810,2431,76,348,4,38730,2164,908,3184,...,77,610,0,897,35,732,1,619,357,276


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,36709,18152,52568,5710,7763,130,1242877,106479,25588,210697,...,4185,69980,0,96652,1179,62944,8,1498,16925,8472
2020-11-29,36731,18481,53204,5794,7763,130,1249843,107364,25592,216998,...,4267,70151,0,96652,1179,63834,8,1504,16983,8472
2020-11-30,36788,18849,53809,5873,7851,130,1257227,108442,25602,221692,...,4357,70337,0,97326,1179,64987,8,1510,16998,8482


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2597,2617,2634,2661,2686,2704,2716,2735,2751,2780
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,609,622,626,633,650,667,671,679,683,689
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3712,3783,3829,3872,3936,3967,4004,4078,4133,4223
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1201,1214,1224,1239,1248,1263,1273,1279,1296,1316
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7478,7504,7539,7597,7644,7718,7735,7757,7785,7814


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,244993,31444,0,322774,155026,1200624,225283,112581,34670,49,...,363466,1213577,192087,4033,1538,233617,160634,45845,404999,31928
2020-11-29,247229,32065,0,325995,156247,1215455,228772,112581,35251,49,...,366518,1225118,193809,4100,1544,235942,162700,46997,409054,32489
2020-11-30,249524,32576,0,326817,157359,1230264,232905,117295,35654,49,...,374493,1238752,195706,4172,1544,237835,165019,47842,411730,33305


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-28        2735    8733    1173  1179   2922     696   1165    6301   
2020-11-29        2751    8820    1175  1188   2946     700   1173    6366   
2020-11-30        2780    8890    1178  1196   2997     702   1178    6430   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-28         1768     1022  ...    1273    323     1896      429   
2020-11-29         1772     1029  ...    1287    330     1933      438   
2020-11-30         1779     1034  ...    1303    331     2000      430   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-28           1780  1570  1037          0      417    396  
2020-11-29           1812  1606  1046          0      428    396  
2020-11-30           1888  1644  1070          0      441    397  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,3572,121,0,6624,2449,19116,2983,4961,763,0,...,4541,21788,863,67,23,4054,2703,718,3464,215
2020-11-29,3577,121,0,6634,2470,19151,3003,4961,770,0,...,4554,21843,868,67,23,4058,2703,729,3487,215
2020-11-30,3578,121,0,6639,2502,19173,3037,5020,772,0,...,4602,21896,871,69,23,4062,2774,735,3494,215


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-28          42      98      10   17     40      19     41     119   
2020-11-29          42      98      11   17     40      19     42     121   
2020-11-30          42      98      11   17     40      19     42     121   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-28           50       24  ...       6      7       10        1   
2020-11-29           50       24  ...       6      7       10        1   
2020-11-30           50       24  ...       6      7       10        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-28              6     2     4          0        8      1  
2020-11-29              6     2     4          0        8      1  
2020-11-30              6     2     4          0        8      1  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.002771,0.015037,0.013023,0.000000,0.005264,0.0,0.004333,0.009430,0.000215,0.017229,...,0.039223,0.002827,0.0,0.002325,0.001494,0.018165,0.0,0.005587,0.001138,0.011118
2020-11-29,0.005417,0.022696,0.012424,0.015431,0.001061,0.0,0.003843,0.008788,0.000359,0.014681,...,0.037198,0.002090,0.0,0.002752,0.001491,0.020698,0.0,0.007870,0.001080,0.000000
2020-11-30,0.006124,0.014804,0.011895,0.004917,0.002384,0.0,0.004036,0.002642,0.000358,0.009825,...,0.024668,0.003074,0.0,0.003469,0.002978,0.024669,0.0,0.006431,0.002215,0.013032


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.006897,0.020752,0.008853,0.0,0.008772,0.0,0.002774,0.014833,0.000000,0.045738,...,0.013514,0.001647,NaN,0.004505,0.0,0.021771,0.0,0.001629,0.0,0.000000
2020-11-29,0.006279,0.013977,0.007104,0.0,0.002899,0.0,0.003940,0.009901,0.001103,0.028827,...,0.013333,0.000000,NaN,0.002242,0.0,0.018466,0.0,0.003252,0.0,0.000000
2020-11-30,0.006239,0.015038,0.008714,0.0,0.005780,0.0,0.006680,0.010271,0.000000,0.025443,...,0.013158,0.003289,NaN,0.003356,0.0,0.020921,0.0,0.003241,0.0,0.003636


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.011407,0.022360,0.011974,0.000000,0.008575,0.0,0.006169,0.012649,0.000117,0.036635,...,0.018992,0.002866,NaN,0.002593,0.007692,0.015881,0.0,0.004695,0.004749,0.000236
2020-11-29,0.000599,0.018125,0.012099,0.014711,0.000000,0.0,0.005605,0.008311,0.000156,0.029906,...,0.019594,0.002444,NaN,0.000000,0.000000,0.014140,0.0,0.004005,0.003427,0.000000
2020-11-30,0.001552,0.019912,0.011371,0.013635,0.011336,0.0,0.005908,0.010041,0.000391,0.021632,...,0.021092,0.002651,NaN,0.006973,0.000000,0.018062,0.0,0.003989,0.000883,0.001180


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,0.008725,0.021705,NaN,0.012980,0.008778,0.012693,0.019597,0.000000,0.014633,0.0,...,0.018923,0.005247,0.010750,0.006991,0.011177,0.013769,0.015597,0.017737,0.013699,0.004878
2020-11-29,0.009127,0.019749,NaN,0.009979,0.007876,0.012353,0.015487,0.000000,0.016758,0.0,...,0.008397,0.009510,0.008965,0.016613,0.003901,0.009952,0.012862,0.025128,0.010012,0.017571
2020-11-30,0.009283,0.015936,NaN,0.002522,0.007117,0.012184,0.018066,0.041872,0.011432,0.0,...,0.021759,0.011129,0.009788,0.017561,0.000000,0.008023,0.014253,0.017980,0.006542,0.025116


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-28      0.006996  0.015111  0.001708  0.005115  0.011773  0.002882   
2020-11-29      0.005850  0.009962  0.001705  0.007634  0.008214  0.005747   
2020-11-30      0.010542  0.007937  0.002553  0.006734  0.017312  0.002857   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-28      0.010408  0.009776  0.001700  0.006897  ...  0.003943   
2020-11-29      0.006867  0.010316  0.002262  0.006849  ...  0.010998   
2020-11-30      0.004263  0.010053  0.003950  0.004859  ...  0.012432   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-28      0.009375  0.007974  0.007042   0.003382  0.014867  0.000000   
2020-11-29      0.021672  0.019515  0.020979   0.017978  0.022930  0.008679   
2020-11-30      0.003030  0.034661 -0.018265   0.041943  0.023661  0.022945   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-28            NaN  0.004819  0.000000  
2020-11-29            NaN  0.026379  0.000000  
2020-11-30            NaN  0.030374  0.002525  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,0.00000,0.016807,NaN,0.005464,0.005337,0.001414,0.002015,0.000000,0.000000,NaN,...,0.003314,0.004379,0.016490,0.000000,0.0,0.002473,0.000000,0.008427,0.008736,0.0
2020-11-29,0.00140,0.000000,NaN,0.001510,0.008575,0.001831,0.006705,0.000000,0.009174,NaN,...,0.002863,0.002524,0.005794,0.000000,0.0,0.000987,0.000000,0.015320,0.006640,0.0
2020-11-30,0.00028,0.000000,NaN,0.000754,0.012955,0.001149,0.011322,0.011893,0.002597,NaN,...,0.010540,0.002426,0.003456,0.029851,0.0,0.000986,0.026267,0.008230,0.002007,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock   Butler   Calhoun   
2020-11-28         0.0     0.0     0.0  0.0    0.0     0.0  0.00000  0.000000   
2020-11-29         0.0     0.0     0.1  0.0    0.0     0.0  0.02439  0.016807   
2020-11-30         0.0     0.0     0.0  0.0    0.0     0.0  0.00000  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-28          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-11-29          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-11-30          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-28            0.0   0.0   0.0        NaN      0.0    0.0  
2020-11-29            0.0   0.0   0.0        NaN      0.0    0.0  
2020-11-30            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.003334,0.016971,0.013330,0.006225,0.005785,0.001361,0.005054,0.010464,0.000311,0.018487,...,0.034752,0.002468,0.0,0.002838,0.003609,0.020985,7.602531e-50,0.005339,0.001248,0.010709
2020-11-29,0.004375,0.019833,0.012877,0.010828,0.003423,0.000681,0.004449,0.009626,0.000335,0.016584,...,0.035975,0.002279,0.0,0.002795,0.002550,0.020842,3.801265e-50,0.006604,0.001164,0.005354
2020-11-30,0.005249,0.017319,0.012386,0.007872,0.002903,0.000340,0.004242,0.006134,0.000347,0.013204,...,0.030321,0.002677,0.0,0.003132,0.002764,0.022756,1.900633e-50,0.006518,0.001689,0.009193


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.005722,0.020233,0.008803,8.138373e-07,0.006015,0.000010,0.004793,0.013792,2.575676e-13,0.042729,...,0.010027,0.001272,NaN,0.004425,2.570094e-28,0.018972,0.0,0.002069,0.000027,0.001427
2020-11-29,0.006000,0.017105,0.007953,4.069187e-07,0.004457,0.000005,0.004367,0.011847,5.512679e-04,0.035778,...,0.011680,0.000636,NaN,0.003334,1.285047e-28,0.018719,0.0,0.002661,0.000013,0.000713
2020-11-30,0.006120,0.016071,0.008334,2.034593e-07,0.005119,0.000003,0.005523,0.011059,2.756340e-04,0.030610,...,0.012419,0.001963,NaN,0.003345,6.425235e-29,0.019820,0.0,0.002951,0.000007,0.002175


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-28,0.006747,0.022346,0.012116,0.006474,0.009847,0.001489,0.006548,0.017104,0.000351,0.036626,...,0.020700,0.002568,NaN,0.003144,0.006309,0.013463,4.454608e-52,0.005040,0.003771,0.003635
2020-11-29,0.003673,0.020235,0.012107,0.010593,0.004924,0.000744,0.006077,0.012708,0.000254,0.033266,...,0.020147,0.002506,NaN,0.001572,0.003154,0.013801,2.227304e-52,0.004523,0.003599,0.001818
2020-11-30,0.002613,0.020074,0.011739,0.012114,0.008130,0.000372,0.005992,0.011374,0.000322,0.027449,...,0.020620,0.002579,NaN,0.004273,0.001577,0.015932,1.113652e-52,0.004256,0.002241,0.001499


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,0.007955,0.021556,NaN,0.012956,0.009638,0.012141,0.020923,0.010001,0.015451,1.469986e-78,...,0.015005,0.005772,0.015322,0.017413,0.006030,0.011521,0.017453,0.019868,0.011457,0.013998
2020-11-29,0.008541,0.020653,NaN,0.011467,0.008757,0.012247,0.018205,0.005000,0.016104,7.349931e-79,...,0.011701,0.007641,0.012143,0.017013,0.004966,0.010737,0.015157,0.022498,0.010735,0.015784
2020-11-30,0.008912,0.018295,NaN,0.006994,0.007937,0.012215,0.018136,0.023436,0.013768,3.674965e-79,...,0.016730,0.009385,0.010966,0.017287,0.002483,0.009380,0.014705,0.020239,0.008638,0.020450


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-28      0.006579  0.011279  0.001464  0.005657  0.009310  0.002146   
2020-11-29      0.006214  0.010621  0.001584  0.006645  0.008762  0.003947   
2020-11-30      0.008378  0.009279  0.002069  0.006690  0.013037  0.003402   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-28      0.006537  0.009126  0.004546  0.006141  ...  0.009910   
2020-11-29      0.006702  0.009721  0.003404  0.006495  ...  0.010454   
2020-11-30      0.005482  0.009887  0.003677  0.005677  ...  0.011443   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-28      0.017265  0.015409  0.015803   0.032855  0.022072  0.009229   
2020-11-29      0.019468  0.017462  0.018391   0.025416  0.022501  0.008954   
2020-11-30      0.011249  0.026062  0.000063   0.033680  0.023081  0.015949   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-28           -1.0  0.027999  0.008112  
2020-11-29           -1.0  0.027189  0.004056  
2020-11-30           -1.0  0.028781  0.003291  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-28,0.002679,0.015711,NaN,0.004717,0.004325,0.002528,0.006642,0.002572,0.001395,NaN,...,0.005693,0.004929,0.015469,0.012334,2.064232e-09,0.003635,0.000883,0.009534,0.011160,0.006486
2020-11-29,0.002040,0.007856,NaN,0.003113,0.006450,0.002179,0.006673,0.001286,0.005285,NaN,...,0.004278,0.003727,0.010631,0.006167,1.032116e-09,0.002311,0.000442,0.012427,0.008900,0.003243
2020-11-30,0.001160,0.003928,NaN,0.001934,0.009703,0.001664,0.008998,0.006589,0.003941,NaN,...,0.007409,0.003077,0.007044,0.018009,5.160579e-10,0.001648,0.013354,0.010329,0.005454,0.001621


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-11-28      0.006316  0.010417  0.000054 -0.000188  0.008541   
2020-11-29      0.003158  0.005208  0.050027 -0.000094  0.004270   
2020-11-30      0.001579  0.002604  0.025014 -0.000047  0.002135   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-28      3.241246e-07  2.910383e-12  0.005768  0.003853  0.000521  ...   
2020-11-29      1.620623e-07  1.219512e-02  0.011288  0.001927  0.000260  ...   
2020-11-30      8.103115e-08  6.097561e-03  0.005644  0.000963  0.000130  ...   

Province_State   Wyoming                                          \
Admin2              Park    Platte  Sheridan Sublette Sweetwater   
2020-11-28      0.007823  0.013549  0.000139      0.0   0.000488   
2020-11-29      0.003911  0.006775  0.000070      0.0   0.000244   
2020-11-30      0.001956  0.003387  0.000035      0.0   0.000122   

Province_State                                                                 
Admin2                 Teton         Uinta Unassigned  Washakie        Weston  
2020-11-28      1.490116e-08  1.589494e-07  -0.972376  0.008929 -4.922185e-07  
2020-11-29      7.450581e-09  7.947468e-08  -0.972376  0.004464 -2.422029e-07  
2020-11-30      3.725290e-09  3.973734e-08  -0.972376  0.002232 -1.201479e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201130,AK,31323,NaN,980073,NaN,totalTestsViral,1011396,162.0,725.0,...,1011396,0,3,271d3c383c856d19d56e16fbd4efca0f728d6f5a,0,0,0,0,0,NaN
1,20201130,AL,249524,41501.0,1376324,NaN,totalTestsViral,1584347,1717.0,25338.0,...,1625848,1,668,1d4953d87fd314f6eb3ff436b379ce3b6ff4fab7,0,0,0,0,0,NaN
2,20201130,AR,157359,19303.0,1545401,NaN,totalTestsViral,1683457,1063.0,8937.0,...,1702760,32,94,95f0301b346ba586616c1c5d7e017e2f206cc369,0,0,0,0,0,NaN
3,20201130,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,007ab4ef8da5c0fee99b382ab79ad5114977ae2c,0,0,0,0,0,NaN
4,20201130,AZ,326817,10077.0,1936949,NaN,totalTestsPeopleViral,2253689,2513.0,25786.0,...,2263766,5,218,bfdf34bd19d13ae6ae7247ce5af2d19d2cf4b941,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201130,AK,31323.0,NaN,980073.0,NaN,totalTestsViral,1011396.0,162.0,725.0,...,1011396,0,3,271d3c383c856d19d56e16fbd4efca0f728d6f5a,0,0,0,0,0,NaN
1,20201130,AL,249524.0,41501.0,1376324.0,NaN,totalTestsViral,1584347.0,1717.0,25338.0,...,1625848,1,668,1d4953d87fd314f6eb3ff436b379ce3b6ff4fab7,0,0,0,0,0,NaN
2,20201130,AR,157359.0,19303.0,1545401.0,NaN,totalTestsViral,1683457.0,1063.0,8937.0,...,1702760,32,94,95f0301b346ba586616c1c5d7e017e2f206cc369,0,0,0,0,0,NaN
3,20201130,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,007ab4ef8da5c0fee99b382ab79ad5114977ae2c,0,0,0,0,0,NaN
4,20201130,AZ,326817.0,10077.0,1936949.0,NaN,totalTestsPeopleViral,2253689.0,2513.0,25786.0,...,2263766,5,218,bfdf34bd19d13ae6ae7247ce5af2d19d2cf4b941,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-30,AK,31323,NaN,980073,NaN,totalTestsViral,1011396,162.0,725.0,NaN,...,1011396,0,3,271d3c383c856d19d56e16fbd4efca0f728d6f5a,0,0,0,0,0,NaN
2020-11-30,AL,249524,41501.0,1376324,NaN,totalTestsViral,1584347,1717.0,25338.0,NaN,...,1625848,1,668,1d4953d87fd314f6eb3ff436b379ce3b6ff4fab7,0,0,0,0,0,NaN
2020-11-30,AR,157359,19303.0,1545401,NaN,totalTestsViral,1683457,1063.0,8937.0,407.0,...,1702760,32,94,95f0301b346ba586616c1c5d7e017e2f206cc369,0,0,0,0,0,NaN
2020-11-30,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,007ab4ef8da5c0fee99b382ab79ad5114977ae2c,0,0,0,0,0,NaN
2020-11-30,AZ,326817,10077.0,1936949,NaN,totalTestsPeopleViral,2253689,2513.0,25786.0,579.0,...,2263766,5,218,bfdf34bd19d13ae6ae7247ce5af2d19d2cf4b941,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-30,AK,31323.0,NaN,980073.0,NaN,totalTestsViral,1011396.0,162.0,725.0,NaN,...,1011396,0,3,271d3c383c856d19d56e16fbd4efca0f728d6f5a,0,0,0,0,0,NaN
2020-11-30,AL,249524.0,41501.0,1376324.0,NaN,totalTestsViral,1584347.0,1717.0,25338.0,NaN,...,1625848,1,668,1d4953d87fd314f6eb3ff436b379ce3b6ff4fab7,0,0,0,0,0,NaN
2020-11-30,AR,157359.0,19303.0,1545401.0,NaN,totalTestsViral,1683457.0,1063.0,8937.0,407.0,...,1702760,32,94,95f0301b346ba586616c1c5d7e017e2f206cc369,0,0,0,0,0,NaN
2020-11-30,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,007ab4ef8da5c0fee99b382ab79ad5114977ae2c,0,0,0,0,0,NaN
2020-11-30,AZ,326817.0,10077.0,1936949.0,NaN,totalTestsPeopleViral,2253689.0,2513.0,25786.0,579.0,...,2263766,5,218,bfdf34bd19d13ae6ae7247ce5af2d19d2cf4b941,0,0,0,0,0,NaN
